In [36]:
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Load packages for ensemble regression
from sklearn.linear_model import Lasso, Ridge
from xgboost.sklearn import XGBRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import RobustScaler


In [37]:
train_df = pd.read_csv('./Data/training_data.csv')
test_df = pd.read_csv('./Data/testing_data.csv')

# Split data into train x and y
train_x = train_df.drop(['price'], axis=1)
train_y = train_df['price']

# we have logged the price so we need to indicate this
log_price = True

In [38]:
# define a cross validation strategy using cross_val_score
n_folds = 10

def rmse_cv(model):
    # split data into kf|olds
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train_x.values)
    # Compute rmse using cross validation
    rmse = np.sqrt(-cross_val_score(model, train_x.values, train_y, scoring="neg_mean_squared_error", cv=kf))
    return rmse


In [39]:
# First we will loop over different alpha values for lasso regression and see which one gives the best rmse
lasso_alphas = [0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
lasso_rmse = []

# Loop over lasso alphas
for alpha in lasso_alphas:
    lasso = make_pipeline(RobustScaler(), Lasso(alpha=alpha, random_state=1))
    lasso_rmse.append(rmse_cv(lasso).mean())

/Users/michael/miniconda3/envs/kaggle/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.126e+01, tolerance: 2.052e+01
  model = cd_fast.enet_coordinate_descent(
/Users/michael/miniconda3/envs/kaggle/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.142e+01, tolerance: 2.054e+01
  model = cd_fast.enet_coordinate_descent(
/Users/michael/miniconda3/envs/kaggle/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features

In [ ]:
# We can now create a score table
lasso_score_table = pd.DataFrame(lasso_rmse,lasso_alpha,columns=['RMSE'])

# Get the best alpha value 
best_lasso_alpha = lasso_score_table.idxmin()[0]

# Plot the rmse vs alpha values 
plt.semilogx(lasso_alphas, lasso_rmse)
plt.vlines(best_lasso_alpha, plt.ylim()[0], np.max(lasso_rmse), color='k', linewidth=3, label='best alpha')
plt.xlabel('alpha')
plt.ylabel('rmse')

plt.legend()
plt.show()